In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

from torch.nn.functional import mse_loss
import pickle
import torch
import sys

sys.path.append('/home/noam.koren/multiTS/NFT/models/NFT/')
from NFT import NFT

sys.path.append('/home/noam.koren/multiTS/NFT/')
from training_functions import read_all_data_and_print_stats, plot_predictions

sys.path.append('/home/noam.koren/multiTS/NFT/')
from dicts import data_to_num_vars_dict

data = 'air_quality'

NUM_VARS = data_to_num_vars_dict[data]
LOOKBACK = 5
HORIZON = 1
blocks = 1 
layers ='tcn'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if data != 'ecg':
    model_path = f"/home/noam.koren/multiTS/NFT/models/NFT/{data}_model_{LOOKBACK}l_{HORIZON}h_{blocks}b_{layers}.pth"
    data_path = f"/home/noam.koren/multiTS/NFT/data/{data}/{data}_{LOOKBACK}l_{HORIZON}h/"
else:
    n_people = 600
    n_rows = 700
    model_path = f"/home/noam.koren/multiTS/NFT/models/NFT/{data}_model_{LOOKBACK}l_{HORIZON}h_{blocks}b_{n_people}people_{n_rows}rows.pth"
    data_path = f"/home/noam.koren/multiTS/NFT/data/{data}/{data}_{LOOKBACK}l_{HORIZON}h_{n_people}people_{n_rows}rows/"


In [2]:
train_X, train_y, val_X, val_y, test_X, test_y = read_all_data_and_print_stats(data_path=data_path)

shape of:
train: X torch.Size([3991, 5, 13]), y: torch.Size([3991, 1, 13])
val: X torch.Size([850, 5, 13]), y: torch.Size([850, 1, 13])
test: X torch.Size([850, 5, 13]), y: torch.Size([850, 1, 13])

The highest value in train is: 3.5929131507873535
The lowest value in the train is: -2.90398907661438
The mean value in the train is: 1.8822399194817763e-09
The median value in the train is: 0.0

The highest value in val is: 3.53997540473938
The lowest value in the val is: -3.0444834232330322
The mean value in the val is: -0.3231194019317627
The median value in the val is: -0.3649429380893707

The highest value in test is: 3.5664443969726562
The lowest value in test train is: -3.2411751747131348
The mean value in the test is: -0.27183297276496887
The median value in the test is: -0.29178187251091003


In [ ]:
model_object = NFT(
    device=device,
    forecast_length=HORIZON,
    backcast_length=LOOKBACK,
    n_vars=NUM_VARS,
    nb_blocks_per_stack=blocks,
    ).to(device)

model_object.load_state_dict(torch.load(model_path))
model_object.eval()

In [ ]:
train_pred = torch.tensor(model_object.predict(train_X), device=device).float()
val_pred = torch.tensor(model_object.predict(val_X), device=device).float()
test_pred = torch.tensor(model_object.predict(test_X), device=device).float()

In [ ]:
accumulated_train_loss = mse_loss(train_pred, train_y.to(device))
accumulated_val_loss = mse_loss(val_pred, val_y.to(device))
accumulated_test_loss = mse_loss(test_pred, test_y.to(device))

print(f"accumulated losses:\n"
f"train loss = {accumulated_train_loss}\n"
f"val loss = {accumulated_val_loss}\n"
f"test loss = {accumulated_test_loss}")

# train_loss_per_point = (accumulated_train_loss) / (HORIZON * NUM_VARS)
# val_loss_per_point = (accumulated_val_loss) / (HORIZON * NUM_VARS)
# test_loss_per_point = (accumulated_test_loss) / (HORIZON * NUM_VARS)

# print(f"per-point loss:\n"
# f"train loss = {train_loss_per_point}\n"
# f"val loss = {val_loss_per_point}\n"
# f"test loss = {test_loss_per_point}")

accumulated losses:
train loss = 0.01752660423517227
val loss = 0.02373112551867962
test loss = 0.018703021109104156


In [ ]:
plot_predictions(train_y, train_pred.cpu().numpy(), HORIZON, NUM_VARS, 0)
plot_predictions(val_y, val_pred.cpu().numpy(), HORIZON, NUM_VARS, 0)
plot_predictions(test_y, test_pred.cpu().numpy(), HORIZON, NUM_VARS, 0)